<a href="https://colab.research.google.com/github/Aravinda89/multi_label_classification/blob/main/Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai==2.2.5 fastcore==1.3.2

  Using cached fastai-2.2.5-py3-none-any.whl (191 kB)
  Using cached fastcore-1.3.2-py3-none-any.whl (46 kB)
     |████████████████████████████████| 12.8 MB 4.3 MB/s 
ERROR: Cannot install fastai==2.2.5 and fastcore==1.3.2 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested fastcore==1.3.2
    fastai 2.2.5 depends on fastcore<1.4 and >=1.3.8

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/user_guide/#fixing-conflicting-dependencies


In [38]:
!pip install wwf==0.0.10 

In [39]:
pip install nbdev==1.1.12 

In [40]:
!nvidia-smi

Mon Feb 14 09:53:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    58W / 149W |   3302MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [41]:
from torchvision.models import vgg19, vgg16

In [42]:
feat_net = vgg19(pretrained=True).features.cuda().eval()
# feat_net = vgg19(pretrained=True).features.eval()

In [43]:
feat_net

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [44]:
for p in feat_net.parameters():
  p.requries_grad=False

In [45]:
layers = [feat_net[i] for i in [1, 6, 11, 20, 29, 22]]; 
layers

[ReLU(inplace=True),
 ReLU(inplace=True),
 ReLU(inplace=True),
 ReLU(inplace=True),
 ReLU(inplace=True),
 ReLU(inplace=True)]

In [46]:
_vgg_config = {
                'vgg16' : [1, 11, 18, 25, 20],
                'vgg19' : [1, 6, 11, 20, 29, 22]
              }

In [47]:
def _get_layers(arch:str, pretrained=True):
  "Get the layers and arch for a VGG Model (16 and 19 are supported only)"

  feat_net = vgg19(pretrained=pretrained).cuda() if arch.find('9') > 1 else vgg16(pretrained=pretrained).cuda()
  config = _vgg_config.get(arch)

  features = feat_net.features.cuda().eval()
  # features = feat_net.features.eval()
  for p in features.parameters(): p.requires_grad=False
  return feat_net, [features[i] for i in config]

In [48]:
def get_feats(arch:str, pretrained=True):
  "Get the features of an architecture"

  feat_net, layers = _get_layers(arch, pretrained)
  hooks = hook_outputs(layers, detach=False)

  def _inner(x):
    feat_net(x)
    return hooks.stored
  return _inner

In [49]:
feats = get_feats('vgg19')

NameError: ignored

In [ ]:
url = 'https://static.greatbigcanvas.com/images/singlecanvas_thick_none/megan-aroon-duncanson/little-village-abstract-art-house-painting,1162125.jpg'